## Análise de Sentimentos com Transformers

#### Importando dataset da Amazon

In [5]:
from datasets import load_dataset

dataset = load_dataset("amazon_polarity")

README.md:   0%|          | 0.00/6.81k [00:00<?, ?B/s]

train-00000-of-00004.parquet:   0%|          | 0.00/260M [00:00<?, ?B/s]

train-00001-of-00004.parquet:   0%|          | 0.00/258M [00:00<?, ?B/s]

train-00002-of-00004.parquet:   0%|          | 0.00/255M [00:00<?, ?B/s]

train-00003-of-00004.parquet:   0%|          | 0.00/254M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3600000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/400000 [00:00<?, ? examples/s]

In [9]:
from datasets import load_dataset
import pandas as pd

# Carregar o dataset
dataset = load_dataset("amazon_polarity")

# Converter para pandas DataFrame
df = pd.DataFrame(dataset["train"])

# Verificar a distribuição das classes
class_counts = df['label'].value_counts()
print(class_counts)

# Balancear para 10k dados, pegando 5k de cada classe
df_balanced = pd.concat([
    df[df['label'] == 0].sample(n=5000, random_state=42),  # Classe negativa
    df[df['label'] == 1].sample(n=5000, random_state=42)   # Classe positiva
])

# Verificar a nova distribuição
print(df_balanced['label'].value_counts())

# Selecionar as primeiras 10k amostras balanceadas
df_balanced = df_balanced.sample(n=10000, random_state=42)

label
1    1800000
0    1800000
Name: count, dtype: int64
label
0    5000
1    5000
Name: count, dtype: int64
         label                        title  \
3071567      1  Road to getting things done   
3551226      0         The Weight Loss Cure   
1551763      0         Royal Velvet Pillows   
3021400      0       Great Book For Kiddies   
2306845      0                   Digimortal   

                                                   content  
3071567  I am a great believer in valueing employees an...  
3551226  This would be a good book. Good Ideas if every...  
1551763  These pillows were over rated and the descript...  
3021400  Robert Sawyer's books have featured an assortm...  
2306845  You know, Fear Factory used to be my favorite ...  


In [10]:
df_balanced

,label,title,content
3071567,1,Road to getting things done,I am a great believer in valueing employees an...
3551226,0,The Weight Loss Cure,This would be a good book. Good Ideas if every...
1551763,0,Royal Velvet Pillows,These pillows were over rated and the descript...
3021400,0,Great Book For Kiddies,Robert Sawyer's books have featured an assortm...
2306845,0,Digimortal,"You know, Fear Factory used to be my favorite ..."
...,...,...,...
15604,1,Good,This is a nice trainer but it's noisier than e...
68917,1,The autobiography of an ex-coloured man,Would recommend this book to a lot of people a...
1365870,1,Wonderful Book! Couln't put it down!!!!!!!!!!!,This is one of the best books I have ever read...
3284310,0,lasted 1 week,I bought this item based on the pivoting plugs...
